# Свёрточные коды

## Терминология

Линейный древовидный $(n_0, k_0)$ код над полем $GF(q)$ с конечной длиной кодового ограничения называется свёрточным кодом. На каждом этапе кодирования кодер свёрточного кода может хранить $m$ ранее поступивших блоков из $k_0$ символов. Скорость кода так же равна $\frac{k_0}{n_0}$. Объём памяти кодера равна $K = k_0 \cdot m$.

Кодер тем или иным образом на каждом этапе преобразует блок из $K$ символов в блок из $n_0$ символов. Поскольку рассматриваемый нами код без памяти, то удобно такой код представить в виде $n_0$ многочленов степени $m$. В таком случае, по аналогии с линейными блоковыми кодами, можно ввести понятие порождающей матрицы многочленов $\underset{k_0 \times n_0}{G(x)}$.

### Катастрофические коды

Катастрофическим кодом называется код, отображающий информационные последовательности бесконечного веса в кодовые последовательности конечного веса. Ошибки в канале для катастрофического кода могут привести к бесконечному числу ошибок при декодировании.

Код не является катастрофическим тогда и только тогда, когда в графе переходов нет петель нулевого веса. Код не является катастрофическим тогда и только тогда, когда НОД определителей всех $k_0 \times k_0$ подматриц $G(x)$ равен $x^s$, где $s \ge 0$.

Примеры:

* $G(x) = (x^2 + 1, x^2 + x + 1)$ является некатастрофическим кодом.
* $G(x) = (\frac{x^2 + 1}{x^2 + x + 1}, 1)$ эквивалентен предыдущему.
* $G(x) = (x^2 + 1, x^2 + 1)$ является катастрофическим кодом.

### Минимальное расстояние

$i$-ым минимальным расстоянием кода $d_i^*$ называется минимально расстояние Хэмминга для любых последовательностей из $i$ кадров, отличающихся начальным кадром. $d_{m+1}^*$ называется минимальным расстоянием кода.

Последовательность $\{ d_i^* \}$ называется дистанционным профилем кода. Минимальным свободным расстоянием кода называется $d_{free} = \max_i{d_i^*}$.

Минимальное расстояние кода можно так же найти при помощи производящей функции (см. ниже).

## Кодирование

Ниже приведён пример кодера кода с порождающей матрицей $G(x) = (x^3+x+1,x^3+x^2+1)$. У данного кода $k_0 = 1$, что соответствует количеству входных стрелок, $n_0 = 2$, что соответствует количеству выходных стрелок, $m = 3$, что соответствует количеству ячеек памяти, а скорость равна $R = \frac{1}{2}$. При кодировании на каждой итерации кодер сдвигает своё положение на $1$ ячейку.

![mem](convolutional-coder.png)

In [1]:
from tree.convolutional import encode_convolutional
from algebraic.binary import binary_array

g = binary_array([[[1, 0, 1, 1], [1, 1, 0, 1]]])
x = binary_array([0, 0, 1, 0, 1, 1, 0, 1, 0])
y = encode_convolutional(g, x)
print("y =", y)

y = [[0, 1], [1, 0], [1, 0], [0, 0], [0, 1], [1, 0], [1, 0]]


In [2]:
g = binary_array([
    [[1, 0, 1, 1], [1, 1, 0, 1], [1, 1, 0, 1]],
    [[1, 0, 0, 1], [0, 1, 1, 0], [0, 1, 1, 0]]
])
x = binary_array([[1, 1, 0, 1, 1, 1, 0], [0, 0, 1, 1, 0, 1, 0]])
y = encode_convolutional(g, x)
print("y =", y)

y = [[1, 0, 0], [0, 0, 0], [0, 1, 1], [1, 0, 0], [1, 0, 0]]


### Граф переходов

По аналогии с линейными блоковыми кодами, для которых вводилась матрица переходов, для свёрточных кодов можно ввести граф переходов, соответствующий некоторому конечному автомату. В качестве состояний будем брать все возможные вектора размерности $m$ над полем $GF(q)$. В качестве переходов — буквально переходы векторов друг в друга, как если бы они двигались по кодеру (т.е. символьный сдвиг вектора вправо), а в качестве меток на рёбрах — новый символ слева. Также каждому ребру в этом конечном автомате можно соотнести, собственно, $n_0$ кодовых символов.

Ниже представлен пример такого графа переходов для кода $G(x) = (x^3+x+1,x^3+x^2+1)$ из предыдущего примера (красный цвет ребра означает метку $1$, чёрный цвет ребра означает метку $0$, на самих рёбрах как раз написаны кодовые символы).

![mem](convolutional-graph.png)

## Декодирование

Декодирование производится аналогично кодированию, но декодер имеет $n_0$ входных стрелок и $k_0$ выходных стрелок.

## Эффективность

Вероятность ошибки декодирования кода определяется числом кодовых слов различного веса (напомним, что вес вектора это количество позиций с ненулевыми значениями). Тогда сначала давайте оценим это число для каждого состояния рекурсивно. Для каждой вершины составим функцию $g_v(D)$, где коэффициент перед $D^i$ показывает число слов веса $i$ (вершины пронумеруем лексикографически). Пример для кода $G(x) = (x^3+x+1,x^3+x^2+1)$ (см. картинки выше):

\begin{equation*}
\begin{gathered}
g_0(D) = g_1(D) \cdot D^2 + \textcolor{red}{g_0(D) \cdot D^0} \\
g_1(D) = g_3(D) \cdot D^1 + g_2(D) \cdot D^1 \\
g_2(D) = g_4(D) \cdot D^1 + g_5(D) \cdot D^1 \\
g_3(D) = g_6(D) \cdot D^2 + g_7(D) \cdot D^1 \\
g_4(D) = \textcolor{red}{g_0(D) \cdot D^2} + g_1(D) \cdot D^0 \\
g_5(D) = g_2(D) \cdot D^1 + g_3(D) \cdot D^1 \\
g_6(D) = g_4(D) \cdot D^1 + g_5(D)\cdot  D^1 \\
g_7(D) = g_7(D) \cdot D^2 + g_6(D) \cdot D^0
\end{gathered}
\end{equation*}

Производящая функция это степенной ряд, коэффициенты которого равны числу кодовых слов различного веса, выходящих из нулевого состояния и возвращающихся в него. Её можно вычислить как $T(D) = \frac{g_0'(D)}{g_0}$, где при вычислении $g_0'(D)$ рекурсивный вызов не осуществляется и $g_0(D) = 0$ (закрашено красным).

Для данного кода производящая функция равна:

$$-\frac{D^{10}-2D^6}{2D^6-5D^2+1} = 2D^\textcolor{blue}{6} + 10D^8 + 49D^{10} + \mathcal{O}(D^{12})$$

Также из производящей функции можно найти минимальное расстояние кода. Оно равно минимальной степени многочленного ряда производящей функции с ненулевым коэффициентом (выделено синим цветом).